# SSD512 Inference

In [1]:
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt
import copy
from PIL import Image
import os
import random

from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms



Using TensorFlow backend.


In [2]:
# nastaveni modelu a nacteni vah
K.clear_session() 

img_height = 512
img_width = 512

model = ssd_512(image_size=(img_height, img_width, 3),
                n_classes=13,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
               two_boxes_for_ar1=True,
               steps=[8, 16, 32, 64, 128, 256, 512],
               offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
               clip_boxes=False,
               variances=[0.1, 0.1, 0.2, 0.2],
               normalize_coords=True,
               subtract_mean=[123, 117, 104],
               swap_channels=[2, 1, 0],
               confidence_thresh=0.5,
               iou_threshold=0.45,
               top_k=200,
               nms_max_output_size=400)




weights_path = 'training_summaries/ssd512/ssd512_coralclef2020_epoch-200.h5'

model.load_weights(weights_path, by_name=True)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [3]:
# nacteni obrazku a anotaci
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path = None)
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path = None)



CoralClef_2020_images_dir      = '../CoralClef2020/training_set_2020/'


CoralClef_2020_annotations_train      =  '../CoralClef2020/annotations/train_annot.json'
CoralClef_2020_annotations_val        =  '../CoralClef2020/annotations/validation_annot.json'



classes = ['c_hard_coral_branching', 'c_hard_coral_submassive', 'c_hard_coral_boulder',
           'c_hard_coral_encrusting', 'c_hard_coral_table', 'c_hard_coral_foliose', 'c_hard_coral_mushroom',
           'c_soft_coral', 'c_soft_coral_gorgonian', 'c_sponge', 'c_sponge_barrel', 'c_fire_coral_millepora',
           'c_algae_macro_or_leaves']


train_dataset.parse_json(images_dirs=[CoralClef_2020_images_dir],
                         annotations_filenames=[CoralClef_2020_annotations_train],
                         ground_truth_available=True,
                         include_classes='all',
                         verbose = True,
                         ret=True
                        )

val_dataset.parse_json(images_dirs=[CoralClef_2020_images_dir],
                       annotations_filenames=[CoralClef_2020_annotations_val],
                       ground_truth_available=True,
                       include_classes='all',
                       verbose = True,
                       ret=True
                      )


resize = Resize(height=img_height, width=img_width)

train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Trenovaci mnozina:\t{:>6}".format(train_dataset_size))
print("Validacni mnozina:\t{:>6}".format(val_dataset_size))

Processing 'validation_annot.json': 100%|███████████████████████████████████████████| 69/69 [00:00<00:00, 13786.54it/s]
Trenovaci mnozina:	   371
Validacni mnozina:	    69


In [4]:
colors = [(1.0, 0.9230769230769231, 0.0),
        (1.0, 0.46153846153846156, 0.0),
        (0.0, 0.3076923076923075, 1.0),
        (1.0, 0.0, 0.9230769230769234),
        (0.1538461538461533, 0.0, 1.0),
        (0.6153846153846154, 1.0, 0.0),
        (0.0, 1.0, 0.30769230769230793),
        (0.0, 0.7692307692307692, 1.0),
        (0.0, 1.0, 0.7692307692307692),
        (1.0, 0.0, 0.0),
        (0.15384615384615374, 1.0, 0.0),
        (0.6153846153846159, 0.0, 1.0),
        (1.0, 0.0, 0.4615384615384617),
        (1.0, 0.9230769230769231, 0.1)]

classes = ['hard coral branching', 'hard coral submassive', 'hard coral boulder',
           'hard coral encrusting', 'hard coral table', 'hard coral foliose', 'hard coral mushroom',
           'soft coral', 'soft coral gorgonian', 'sponge', 'sponge barrel', 'fire coral millepora',
           'algae macro or leaves']

## Detekce na jednotlivých obrázcích

In [ ]:
IMAGE_DIR = 'images/'
SAVE_DIR = 'images_out/'

# nacteni obrazku

file_names = os.listdir(IMAGE_DIR)
file_name = file_names[random.randint(0, len(file_names)-1)]
file_name = file_names[3]
img = np.array(Image.open(IMAGE_DIR + file_name), dtype=np.uint8) 
img2 = resize(img)
reshape_image = img2.reshape([1,512,512,3])

# detekce
y_pred = model.predict(reshape_image)
confidence_threshold = 0.5
y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]

# zmena velikosti 
y_pred_resized = copy.deepcopy(y_pred_thresh)
for r in y_pred_resized[0]:
    r[2] = r[2] * (img.shape[1]/img_width)
    r[3] = r[3] * (img.shape[0]/img_height)
    r[4] = r[4] * (img.shape[1]/img_width)
    r[5] = r[5] * (img.shape[0]/img_height)

fig = plt.figure(figsize=(16, 16))
plt.axis('off')
plt.imshow(img)

current_axis = plt.gca()

# zobrazeni vysledku
for box in y_pred_resized[0]:
    class_id = box[0]
    confidence = box[1]
    xmin = box[2]
    ymin = box[3]
    xmax = box[4]
    ymax = box[5]
    label = '{}: {:.2f}'.format(classes[int(class_id)-1], confidence)
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=colors[int(box[0])-1], fill=False, linewidth=4))#, linestyle='--'  
    current_axis.text(xmin, ymin, label, size='x-large', color='black', bbox={'facecolor':colors[int(box[0])-1], 'alpha':1.0})


fig.savefig(SAVE_DIR + file_name)

## Detekce na validační a trénovací množině

In [ ]:
dataset = val_dataset
#dataset = train_dataset

progbar = tf.keras.utils.Progbar(dataset.get_dataset_size())
k = 0

image_ids = list(range(dataset.get_dataset_size()))

for ind in image_ids:
    progbar.update(k)
    # nacteni obrazku a anotaci
    original_labels = dataset.labels[ind]
    file_names = dataset.filenames[ind]
             
    img = np.array(Image.open(file_names), dtype=np.uint8) 
    img2, labels  = resize(img, original_labels)
    images = img2.reshape([1,512,512,3])
    labels = [np.array(labels)]
    small_labels = [np.array(original_labels)]
    
    # detekce
    y_pred = model.predict(images)
    confidence_threshold = 0.5
    y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
    
    # zmena velikosti
    y_pred_resized = copy.deepcopy(y_pred_thresh)
    for r in y_pred_resized[0]:
        r[2] = r[2] * (img.shape[1]/img_width)
        r[3] = r[3] * (img.shape[0]/img_height)
        r[4] = r[4] * (img.shape[1]/img_width)
        r[5] = r[5] * (img.shape[0]/img_height)
    
    fig = plt.figure(figsize=(16, 16))
    plt.axis('off')
    plt.imshow(img)

    current_axis = plt.gca()
    
    # zobrazeni vysledku
    for box in y_pred_resized[0]:
        class_id = box[0]
        confidence = box[1]
        xmin = box[2]
        ymin = box[3]
        xmax = box[4]
        ymax = box[5]
        label = '{}: {:.2f}'.format(classes[int(class_id)-1], confidence)
        current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=colors[int(box[0])-1], fill=False, linewidth=4))#, linestyle='--'  
        current_axis.text(xmin, ymin, label, size='x-large', color='black', bbox={'facecolor':colors[int(box[0])-1], 'alpha':1.0})
    
    
    
    # zobrazeni graund truth dat
    fig = plt.figure(figsize=(16, 16))
    plt.axis('off')
    plt.imshow(img)

    current_axis = plt.gca()
    for box in original_labels:
        class_id = box[0]
        xmin = box[1]
        ymin = box[2]
        xmax = box[3]
        ymax = box[4]
        label = '{}'.format(classes[int(class_id)-1])
        current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=colors[int(box[0])-1], fill=False, linewidth=4))  
        current_axis.text(xmin, ymin, label, size='x-large', color='black', bbox={'facecolor':colors[int(box[0])-1], 'alpha':1.0})
    
    k = k + 1